In [99]:
# edges are encoded at 8 bit bytes. The first bit encodes flipped or not. The last 4 bits encode slice (0, 1, 2) and number within the slice (0, 1, 2, 3)

edges   = [0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B]
corners = [0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07]
cube    = bytes(edges + corners)


In [100]:
def generate_new(members, generators):
    candidates = dict()
    for name, cube in members.items():
        candidates.update({name + n: bytes([cube[0x0F & generator[i]] ^ (generator[i] & 0xF0)  for i in range(0,12)] +
                                      [(cube[12 + (0x0F & generator[j])] & 0x0F) + (((cube[12 + (0x0F & generator[j])] & 0xF0) + (generator[j] & 0xF0))%0xC0)    for j in range(12, 20)])
                           for n, generator in generators.items()} )
        
            
    return candidates
    
    

In [101]:
# Define base permutations (clockwise quarter turns in each face - cubies numbered from 0
moves = {'R': bytes([0x00, 0x01, 0x06, 0x07, 0x04, 0x05, 0x03, 0x02, 0x08, 0x09, 0x0A, 0x0B]+ [0x00, 0x01, 0x07, 0x06, 0x04, 0x05, 0x02, 0x03]),
         'L': bytes([0x04, 0x05, 0x02, 0x03, 0x01, 0x00, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B]+ [0x05, 0x04, 0x02, 0x03, 0x00, 0x01, 0x06, 0x07]),
         'F': bytes([0x00, 0x01, 0x02, 0x03, 0x04, 0x09, 0x08, 0x07, 0x05, 0x06, 0x0A, 0x0B]+ [0x00, 0x86, 0x02, 0x84, 0x41, 0x05, 0x43, 0x07]),
         'B': bytes([0x00, 0x01, 0x02, 0x03, 0x0B, 0x05, 0x06, 0x0A, 0x08, 0x09, 0x04, 0x07]+ [0x87, 0x01, 0x85, 0x03, 0x04, 0x40, 0x06, 0x42]),
         'U': bytes([0x88, 0x01, 0x8B, 0x03, 0x04, 0x05, 0x06, 0x07, 0x82, 0x09, 0x0A, 0x80]+ [0x44, 0x01, 0x02, 0x47, 0x83, 0x05, 0x06, 0x80]),
         'D': bytes([0x00, 0x8A, 0x02, 0x89, 0x04, 0x05, 0x06, 0x07, 0x08, 0x81, 0x83, 0x0B]+ [0x00, 0x45, 0x46, 0x03, 0x04, 0x82, 0x81, 0x07])
         }

In [116]:
mask =  bytes([0x00, 0x00, 0x00, 0xFF, 0x00, 0x00, 0x00, 0xFF, 0x00, 0x00, 0x00, 0xFF, 0x00, 0x00, 0x00, 0xFF,0x00, 0x00, 0x00, 0xFF])

In [119]:
[cube[i] & mask[i]  for i in range(0,20)]

[0, 0, 0, 3, 0, 0, 0, 7, 0, 0, 0, 11, 0, 0, 0, 3, 0, 0, 0, 7]

In [103]:
squared_moves = generate_new(moves, moves)
three_moves   = generate_new(moves, squared_moves)
four_moves    = generate_new(squared_moves, squared_moves)

   

In [108]:
# Check that four moves brings youback to the start
print(four_moves['UUUU'])
print(four_moves['DDDD'])
print(four_moves['FFFF'])
print(four_moves['RRRR'])
print(four_moves['LLLL'])
print(four_moves['RRRR'])



b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x00\x01\x02\x03\x04\x05\x06\x07'
b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x00\x01\x02\x03\x04\x05\x06\x07'
b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x00\x01\x02\x03\x04\x05\x06\x07'
b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x00\x01\x02\x03\x04\x05\x06\x07'
b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x00\x01\x02\x03\x04\x05\x06\x07'
b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x00\x01\x02\x03\x04\x05\x06\x07'


In [113]:
print(squared_moves['RR'])
print(squared_moves['LL'])
print(squared_moves['FF'])
print(squared_moves['BB'])
print(squared_moves['UU'])
print(squared_moves['DD'])


b'\x00\x01\x03\x02\x04\x05\x07\x06\x08\t\n\x0b\x00\x01\x03\x02\x04\x05\x07\x06'
b'\x01\x00\x02\x03\x05\x04\x06\x07\x08\t\n\x0b\x01\x00\x02\x03\x05\x04\x06\x07'
b'\x00\x01\x02\x03\x04\x06\x05\x07\t\x08\n\x0b\x00\x03\x02\x01\x06\x05\x04\x07'
b'\x00\x01\x02\x03\x07\x05\x06\x04\x08\t\x0b\n\x02\x01\x00\x03\x04\x07\x06\x05'
b'\x02\x01\x00\x03\x04\x05\x06\x07\x0b\t\n\x08\x03\x01\x02\x00\x07\x05\x06\x04'
b'\x00\x03\x02\x01\x04\x05\x06\x07\x08\n\t\x0b\x00\x02\x01\x03\x04\x06\x05\x07'


In [112]:
print(three_moves['RRR'])
print(three_moves['LLL'])
print(three_moves['FFF'])
print(three_moves['BBB'])
print(three_moves['UUU'])
print(three_moves['DDD'])


b'\x00\x01\x07\x06\x04\x05\x02\x03\x08\t\n\x0b\x00\x01\x06\x07\x04\x05\x03\x02'
b'\x05\x04\x02\x03\x00\x01\x06\x07\x08\t\n\x0b\x04\x05\x02\x03\x01\x00\x06\x07'
b'\x00\x01\x02\x03\x04\x08\t\x07\x06\x05\n\x0b\x00\x84\x02\x86C\x05A\x07'
b'\x00\x01\x02\x03\n\x05\x06\x0b\x08\t\x07\x04\x85\x01\x87\x03\x04B\x06@'
b'\x8b\x01\x88\x03\x04\x05\x06\x07\x80\t\n\x82G\x01\x02D\x80\x05\x06\x83'
b'\x00\x89\x02\x8a\x04\x05\x06\x07\x08\x83\x81\x0b\x00FE\x03\x04\x81\x82\x07'
